In [1]:
import ee
import geemap
import sys

sys.path.append('../../')

from src.ee_scripts.ee_utils import *

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
# =============Feature Imports================
table = ee.FeatureCollection("projects/ee-ayang115t/assets/ec_denmark_50km")
denmark_pts = ee.FeatureCollection("projects/ee-ayang115t/assets/Marker_2018")
countries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")
geometry = ee.Geometry.Polygon([[
    [9.099959167786013, 55.421683620075065],
    [9.099959167786013, 55.337420105686874],
    [9.237288269348513, 55.337420105686874],
    [9.237288269348513, 55.421683620075065]
    ]])

# =============Extraction Variables================
random_seed = 1
start_date = '2018-01-01'
end_date = '2019-01-01'
denmark_bounds = countries.filter(ee.Filter.eq('ADM0_NAME', 'Denmark'))
step = 10 # date range for collected averages
pixel_limit = 1 # number of pixels to sample per feature
scale = 100 # resolution (meters) to sample in
dataset_properties = ['Afgkode', 'Afgroede'] # dataset labels to copy to SAR samples-

aoi = denmark_pts.filterBounds(geometry) #subset to investigate

In [ ]:
s1 = generate_s1_averages(start_date, end_date, aoi, step)
sampled_sar = ee.FeatureCollection(aoi.map(sampleSAR)).flatten()

In [22]:
# =============S1-SAR================
def toNatural(img):
    return ee.Image(10.0).pow(img.select('..').divide(10.0)).copyProperties(img, ['system:time_start'])

def toDB(img):
    return ee.Image(img).log10().multiply(10.0).copyProperties(img, ['system:time_start'])

def maskEdge(img):
    mask = img.select(0).unitScale(-25, 5).multiply(255).toByte().connectedComponents(ee.Kernel.rectangle(1, 1), 100)
    return img.updateMask(mask.select(0).abs())

# Filter and preprocess the SAR images
s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterMetadata('instrumentMode', 'equals', 'IW') \
    .filterDate(start_date, end_date) \
    .filterBounds(aoi) \
    .filter(ee.Filter.eq('transmitterReceiverPolarisation', ['VV', 'VH'])) \
    .sort('system:time')

s1 = s1.map(maskEdge)
s1 = s1.map(toNatural)

# Define the date ranges
days = ee.List.sequence(0, ee.Date(end_date).difference(ee.Date(start_date), 'day'), step) \
    .map(lambda d: ee.Date(start_date).advance(d, "day"))
dates = days.slice(0, -1).zip(days.slice(1))

# Extract the SAR dB for each date range
def get_range_average(date_range):
    dstamp = ee.Date(ee.List(date_range).get(0)).format('YYYYMMdd')
    date_range_start = ee.List(date_range).get(0)
    date_range_end = ee.List(date_range).get(1)
    temp_collection = s1.filterDate(date_range_start, date_range_end).mean().select(
        ['VV', 'VH'], 
        [ee.String('VV_').cat(dstamp), ee.String('VH_').cat(dstamp)]
        )
    
    return temp_collection

s1 = dates.map(get_range_average)
s1 = ee.ImageCollection(s1.map(toDB)).toBands()

In [23]:
# =============Extraction Variables================
def sampleSAR(feature):
    code = feature.get('Afgkode')
    desc = feature.get('Afgroede')
    sampled = s1.sample(
        region=feature.geometry(),
        scale=scale,
        numPixels=pixel_limit,
        seed=random_seed,
        geometries=False
    )

    sampled = sampled.map(lambda x: x.set('code', code))
    sampled = sampled.map(lambda x: x.set('crop', desc))

    return sampled

sampled_sar = ee.FeatureCollection(aoi.map(sampleSAR)).flatten()

In [32]:
features = sampled_sar.getInfo()

In [28]:
# Set the center of the map
Map = geemap.Map()
Map.centerObject(aoi.geometry())

# Add layers to the map
# Map.addLayer(aoi, {'color': 'blue'}, 'aoi')
Map.addLayer(sampled_sar,{'color': 'blue'},'samples')
Map.addLayer(s1.clip(aoi), {'bands': ['0_VV_20180101'], 'min': -25, 'max': -1, 'palette': ['black', 'white']}, 'S1')

In [29]:
Map

Map(center=[55.38137122310107, 9.167518561761671], controls=(WidgetControl(options=['position', 'transparent_b…

In [26]:
# =============Export Samples================
task = ee.batch.Export.table.toDrive(
    collection=sampled_sar,
    description='ec_'+str(step)+'days_'+str(scale)+'m_2018_denmark_'+str(pixel_limit)+'per',
    fileNamePrefix='ec_'+str(step)+'days_'+str(scale)+'m_2018_denmark_'+str(pixel_limit)+'per',
    fileFormat='CSV',
    folder='ec_denmark_s1',
)
task.start()


TypeError: Object of type FeatureCollection is not JSON serializable